## mapping binary classification results on brain

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import scipy.io as scio
import os
import tqdm
import seaborn as sns
import matplotlib
from tqdm import trange
import pandas as pd
import itertools
from sklearn.cluster import KMeans
import matplotlib.image as mpimg
from scipy.signal import find_peaks, peak_prominences
import sys
sys.path.append("..")
from sklearn.cluster import KMeans
from Project.overt_reading_process import HS_block_erps
#输出当前路径
import os,sys
from matplotlib.colors import Normalize
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm, ListedColormap
from scipy import stats
# from matplotlib_venn import venn3
from sklearn.metrics import silhouette_score
# from statannotations.Annotator import Annotator
# print(os.getcwd())
# cwd = os.getcwd()
# subjects = [68,69,75,79,82]
# matplotlib.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['axes.unicode_minus'] = False 

subjects = [68,69,75,79,82,83,84,85,86]

path = "/public/DATA/overt_reading/"

In [2]:
subjects = [68,69,75,79,82,83,84,85,86]

path = "/public/DATA/overt_reading/"
# load sig elecs
sig_elecs_HS_lists = {}
for HS in subjects:
    n_std = 5
    subject = f"HS{HS}"
    sig_elecs_HS_lists[subject] =np.load(path +f"/elecs/sig_elecs/HS{HS}_sig_elecs{n_std}.npy",allow_pickle=True).item()
# print(sig_elecs_HS_lists)
# load warped elecs positions
xy_all = {}
for HS in subjects:
    subject = f"HS{HS}"
    xy_all[subject] = scio.loadmat(path +"/elecs/All_elecs/"+ subject + "_warped_elec_pos2D.mat")['elecmatrix']
# print(xy_all['HS86'].shape)(256, 2) 256个电极的坐标x，y
# load erps
HS_BLOCK = {}
for subject in subjects:
   HS_DATA = HS_block_erps.HS_block_process(subject,path)
   HS_BLOCK[f"HS{subject}"] = HS_DATA.get_HSblock()

task_word = HS_block_erps.get_task_word()
# load HS elec information
HS_elec_list = HS_block_erps.get_HS_elec(subjects=subjects,sig_elecs_HS_lists=sig_elecs_HS_lists)

In [6]:
subjects = [68,69,75,79,82,83,84,85,86]

path = "/public/DATA/overt_reading/"
sig_elecs_HS_lists = {}
for HS in subjects:
    n_std = 5
    
    subject = f"HS{HS}"
    sig_elecs_HS_lists[subject] =np.load(path +f"/elecs/sig_elecs/HS{HS}_sig_elecs{n_std}.npy",allow_pickle=True).item()
xy_all = {}
for HS in subjects:
    subject = f"HS{HS}"

    xy_all[subject] = scio.loadmat(path +"/elecs/All_elecs/"+ subject + "_warped_elec_pos2D.mat")['elecmatrix']
#for subject in subjects:
#    HS_DATA = HS_block_erps.HS_block_process(subject,path)
#    HS_BLOCK[f"HS{subject}"] = HS_DATA.get_HSblock()

#task_word = HS_block_erps.get_task_word()
#HS_elec_list = HS_block_erps.get_HS_elec(subjects=subjects,sig_elecs_HS_lists=sig_elecs_HS_lists,task_list=["cue"])
subjects = [68,69,75,79,82,83,84,85,86]
marker_type = {}
marker_type["68"] = "D"
marker_type["69"] = "*"
marker_type["75"] = "s"
marker_type["79"] = "P"
marker_type["82"] = "o"
marker_type["83"] = "v"
marker_type["84"] = "^"
marker_type["85"] = "X"
marker_type["86"] = "p"
color_set_region = {'superiorfrontal': 'grey',
             'superiortemporal': 'purple',
             'inferiorparietal': 'skyblue',
             'rostralmiddlefrontal': 'black',
             'parstriangularis': 'darkgreen',
             'superiorparietal': 'orange',
             'postcentral': 'red',
             'parsopercularis': 'gold',
             'middletemporal': 'greenyellow',
             'inferiortemporal': 'pink',
             'parsorbitalis': 'olive',
             'supramarginal': 'green',
             'lateraloccipital': 'navy',
             'bankssts': 'rosybrown',
             'precentral': 'blue',
             'caudalmiddlefrontal': 'slategrey'}
from matplotlib import cm
import matplotlib as mpl
from matplotlib.cm import ScalarMappable

img = mpimg.imread(path +"/elecs/Brain2D/MNI152_brain2D.png")
# img = np.load(path+"/elecs/Brain2D/MNI152_brain2D.png.npy")

fig, ax = plt.subplots(figsize=(15,15))

ax.imshow(img)

x_coords, y_coords = [],[]
values = []
values_2 = []
marker= []
region_all = []
for i,subject in enumerate(subjects):
    x_coords, y_coords = [],[]
    values = []
    values_2 = []
    marker= []
    
    HS = str(subject)
    print(HS)
    #xy = scio.loadmat(path +f"/elecs/All_elecs/HS{HS}_warped_elec_pos2D.mat")['elecmatrix']
    xy = xy_all[f"HS{HS}"]
    anatomy = scio.loadmat(path+f"/elecs/All_elecs/HS{HS}_elecs_all_warped.mat")['anatomy']
    for elec in range(256):
        elec = int(elec)
        x_coords.append(xy[elec][0])
        y_coords.append(xy[elec][1])
        
        
        flag = 0
        color = color_set_region[anatomy[elec][3][0]]
        marker.append(marker_type[HS])
        region_all.append(anatomy[elec][3][0])
        ax.scatter(x_coords[elec], y_coords[elec],c=color,alpha = 0.1,s=50,marker=marker[elec])
x_coords, y_coords = [],[]
values = []
values_2 = []
marker= []
region = []
for i,img_temp in enumerate(HS_elec_list):
    
    HS,elec = HS_elec_list[i].split("_")
    #xy = scio.loadmat(path +f"/elecs/All_elecs/HS{HS}_warped_elec_pos2D.mat")['elecmatrix']
    xy = xy_all[f"HS{HS}"]
    anatomy = scio.loadmat(path+f"/elecs/All_elecs/HS{HS}_elecs_all_warped.mat")['anatomy']
    elec = int(elec)
    x_coords.append(xy[elec][0])
    y_coords.append(xy[elec][1])
    
    
    flag = 0
    color = color_set_region[anatomy[elec][3][0]]
    marker.append(marker_type[HS])
    region.append(anatomy[elec][3][0])
    ax.scatter(x_coords[i], y_coords[i],c=color,alpha = 0.7,s=100,marker=marker[i])
#加上color bar
# im = plt.scatter(x_coords, y_coords, c=values_2, cmap='brg', alpha = 0.7,s=25*np.array(values),marker=marker)
# for i in range(len(x_coords)):
#     ax.scatter(x_coords[i], y_coords[i],c='k',alpha = 0.7,s=100,marker=marker[i])
ax.grid('off') 
ax.axis('off')

ax.set_xticks([])
ax.set_yticks([])

ax.set_title("All electrodes distribution",size=25)

plt.savefig("/root/pp/covert-reading/Ecog_pretrain/figures/fig_1_elec_distri.pdf",dpi=1200)

68
69
75
79
82
83
84
85
86


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
